In [ ]:
# 1. Cài đặt Unsloth và các thư viện lõi
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

# 2. Cài đặt thư viện hỗ trợ API và Tunneling
!pip install fastapi uvicorn python-multipart pyngrok

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-657ljho0/unsloth_a4eb1b3a74b14ed0805d92094a62281e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-657ljho0/unsloth_a4eb1b3a74b14ed0805d92094a62281e
  Resolved https://github.com/unslothai/unsloth.git to commit 06daf28c8b79782375bb7e17a830b11266407bc9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 140.1 MB/s eta 0:00:00

In [ ]:
import torch
try:
    from unsloth import FastLanguageModel
    print("✅ Unsloth installed successfully.")
except ImportError:
    print("❌ Unsloth installation failed.")

print(f"✅ Torch version: {torch.__version__}")

if torch.cuda.is_available():
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    major_version, minor_version = torch.cuda.get_device_capability()
    if major_version >= 8:
        print("✅ GPU supports bfloat16 (Ampere or newer).")
    else:
        print("ℹ️ GPU does not support bfloat16 (using float16 instead) - This is normal for T4.")
else:
    print("❌ No GPU detected. Please change Runtime to T4 GPU.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
✅ Unsloth installed successfully.
✅ Torch version: 2.9.0+cu126
✅ GPU available: Tesla T4
ℹ️ GPU does not support bfloat16 (using float16 instead) - This is normal for T4.


In [ ]:
from unsloth import FastLanguageModel
import torch
from huggingface_hub import login

# 1. Cấu hình Load Model
max_seq_length = 2048 # Chọn độ dài context window
dtype = None # None để tự động chọn float16 cho T4
load_in_4bit = True # BẮT BUỘC True để chạy mượt trên T4 (tiết kiệm VRAM)

print(f"⏳ Đang tải model 'AnLeDinhMinh/formal-converter-v1.3-cot'...")

# 2. Load Model & Tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "AnLeDinhMinh/formal-converter-v1.3-cot",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 3. Kích hoạt chế độ Inference (Tăng tốc 2x)
FastLanguageModel.for_inference(model)

print("✅ Model loaded & Optimized for Inference!")

🔹 Vui lòng nhập HuggingFace Token của bạn (bắt đầu bằng hf_...):


⏳ Đang tải model 'AnLeDinhMinh/formal-converter-v1.3-cot'...
==((====))==  Unsloth 2025.12.9: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/323M [00:00<?, ?B/s]

Unsloth 2025.12.9 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


✅ Model loaded & Optimized for Inference!


In [ ]:
# Kiểm tra bộ nhớ VRAM và cấu hình
print(f"Model Device: {model.device}")
print(f"Is Loaded in 4-bit: {model.is_loaded_in_4bit}")

# In thử cấu trúc chat template (nếu có sẵn trong tokenizer)
try:
    print("Tokenizer Chat Template:", tokenizer.chat_template[:100] if tokenizer.chat_template else "None")
except:
    print("Tokenizer does not have a default chat template.")

Model Device: cuda:0
Is Loaded in 4-bit: True
Tokenizer Chat Template: {%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
    


In [ ]:
# Cập nhật Prompt để yêu cầu Reasoning cụ thể
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a formal text converter. First, think step-by-step about the context, intent, and appropriate tone in a 'Reasoning' section. Then, provide the final converted text in a 'Result' section.

### Context:
{}

### Intent:
{}

### Input:
{}

### Response:
### Reasoning:
"""

# Cập nhật hàm format
def format_prompt(context, intent, input_text):
    return alpaca_prompt.format(context, intent, input_text)

print("✅ Đã cập nhật Prompt Template để kích hoạt CoT!")

✅ Đã cập nhật Prompt Template để kích hoạt CoT!


In [ ]:
# Dữ liệu giả lập để test
test_context = "Gửi tin nhắn cho sếp xin nghỉ phép"
test_intent = "Muốn xin nghỉ 1 ngày vì bị ốm, cần giọng điệu lịch sự, tôn trọng"
test_input = "sếp ơi nay em bệnh quá nghỉ nha"

# Gọi hàm format
formatted_text = format_prompt(test_context, test_intent, test_input)

print("🔹 PROMPT MẪU SẼ GỬI VÀO MODEL:\n")
print(formatted_text)

🔹 PROMPT MẪU SẼ GỬI VÀO MODEL:

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a formal text converter. First, think step-by-step about the context, intent, and appropriate tone in a 'Reasoning' section. Then, provide the final converted text in a 'Result' section.

### Context:
Gửi tin nhắn cho sếp xin nghỉ phép

### Intent:
Muốn xin nghỉ 1 ngày vì bị ốm, cần giọng điệu lịch sự, tôn trọng

### Input:
sếp ơi nay em bệnh quá nghỉ nha

### Response:
### Reasoning:



In [ ]:
# 1. Chuyển đổi text sang dạng số (Tokenization)
inputs = tokenizer(
    [formatted_text],
    return_tensors = "pt" # Trả về PyTorch tensor
).to("cuda") # Đẩy dữ liệu vào GPU

# 2. Sinh văn bản (Inference)
print("⏳ Model đang suy nghĩ và viết câu trả lời...")
outputs = model.generate(
    **inputs,
    max_new_tokens = 512, # Giới hạn độ dài câu trả lời (đủ cho cả suy luận + kết quả)
    use_cache = True,
)

# 3. Giải mã kết quả (Decoding)
# skip_special_tokens=True để loại bỏ các ký tự kỹ thuật như <eos>
response_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print("\n" + "="*20 + " KẾT QUẢ TỪ MODEL " + "="*20 + "\n")
print(response_text)

⏳ Model đang suy nghĩ và viết câu trả lời...

==================== KẾT QUẢ TỪ MODEL ====================

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a formal text converter. First, think step-by-step about the context, intent, and appropriate tone in a 'Reasoning' section. Then, provide the final converted text in a 'Result' section.

### Context:
Gửi tin nhắn cho sếp xin nghỉ phép

### Intent:
Muốn xin nghỉ 1 ngày vì bị ốm, cần giọng điệu lịch sự, tôn trọng

### Input:
sếp ơi nay em bệnh quá nghỉ nha

### Response:
### Reasoning:
Input is too casual for a professional request to the boss. The phrase 'nghỉ nha' (go ahead and rest) is inappropriate. A more formal way would be to explain the reason for absence and request permission to take a leave day.

### Result:
Em chào Thầy, do em bị cảm lạnh nên không thể đến trường hôm nay. Em xin phép được n

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from transformers import TextIteratorStreamer
from threading import Thread
from fastapi.responses import StreamingResponse
import json

# 1. Khởi tạo App
app = FastAPI()

# 2. Cấu hình CORS (Để Frontend React gọi được API này)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], # Cho phép mọi nguồn truy cập (để test cho dễ)
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 3. Định nghĩa format dữ liệu đầu vào (Request Body)
class RequestBody(BaseModel):
    context: str
    intent: str
    input_text: str

# 4. Định nghĩa Endpoint Streaming
@app.post("/generate")
async def generate_text(body: RequestBody):
    # a. Chuẩn bị Prompt từ input của Client
    prompt = format_prompt(body.context, body.intent, body.input_text)

    # b. Tokenize
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # c. Tạo Streamer (cơ chế hứng từng chữ một)
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # d. Cấu hình các tham số sinh văn bản
    generation_kwargs = dict(
        inputs,
        streamer=streamer,
        max_new_tokens=512,
        use_cache=True
    )

    # e. Chạy model trong một Luồng riêng (Thread) để không chặn Web Server
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    # f. Hàm generator để trả về dữ liệu dần dần cho Client
    def stream_generator():
        for new_text in streamer:
            # Gửi từng mảnh text (token) về cho client ngay lập tức
            # Dùng JSON format để dễ xử lý ở frontend
            yield json.dumps({"text": new_text}) + "\n"

    # g. Trả về StreamingResponse
    return StreamingResponse(stream_generator(), media_type="application/x-ndjson")

print("✅ Đã khởi tạo FastAPI App và Streaming Logic thành công!")

✅ Đã khởi tạo FastAPI App và Streaming Logic thành công!


In [ ]:
import uvicorn
from pyngrok import ngrok
import nest_asyncio

# 1. Patch asyncio (Bắt buộc trên Colab)
nest_asyncio.apply()

# 2. Cấu hình Ngrok
NGROK_TOKEN = ""

ngrok.set_auth_token(NGROK_TOKEN)

# Kill các tunnel cũ để tránh xung đột
ngrok.kill()

# Mở tunnel mới
tunnel = ngrok.connect(8000)

print("\n" + "="*40)
print(f"🚀 PUBLIC URL CỦA BẠN: {tunnel.public_url}")
print("="*40 + "\n")
print("⚠️ Server đang chạy... (Cell này sẽ không dừng lại, đây là điều bình thường)")

# 3. Khởi chạy Server (CÁCH SỬA LỖI Ở ĐÂY)
# Thay vì uvicorn.run(), ta dùng config và await server.serve() để dùng lại loop của Colab
config = uvicorn.Config(app, port=8000)
server = uvicorn.Server(config)
await server.serve()


🚀 PUBLIC URL CỦA BẠN: https://highly-mendy-licenceable.ngrok-free.dev

⚠️ Server đang chạy... (Cell này sẽ không dừng lại, đây là điều bình thường)


INFO:     Started server process [255]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     116.111.185.79:0 - "POST //generate HTTP/1.1" 404 Not Found
INFO:     116.111.185.79:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     116.111.185.79:0 - "OPTIONS /generate HTTP/1.1" 200 OK
INFO:     116.111.185.79:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     116.111.185.79:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     116.111.185.79:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     116.111.185.79:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     116.111.185.79:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     116.111.185.79:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     116.111.185.79:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     116.111.185.79:0 - "OPTIONS /generate HTTP/1.1" 200 OK
INFO:     116.111.185.79:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     116.111.185.79:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     116.111.185.79:0 - "POST /generate HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [255]
